In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import string
import unicodedata

from scipy import sparse
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.util import ngrams
from nltk import pos_tag
from nltk import RegexpParser
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

from nlp import filter_tokens, validate_string, extract_bow_from_column, create_Xy_csv
import pickle



In [31]:
create_Xy_csv("../data/clean_amazon_reviews.csv")

In [32]:

nlp_df = pd.read_csv('../data/df_with_nlp.csv', index_col=0)
X = nlp_df
y = pd.read_csv('../data/work-balance-stars.csv', index_col=0).values

In [13]:
amazon_df = pd.read_csv('../data/clean_amazon_reviews.csv')

In [14]:
nlp_df = pd.read_csv('../data/df_with_nlp.csv', index_col=0)

In [15]:
X = nlp_df
y = amazon_df['work-balance-stars'].values

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [24]:
gbr = GradientBoostingRegressor(learning_rate=0.001,
                                   n_estimators=2000,
                                   min_samples_leaf=5,
                                   max_depth=3,
                                   subsample=0.5,
                               verbose=1)
gbr.fit(X_train, y_train)


      Iter       Train Loss      OOB Improve   Remaining Time 
         1           1.6172           0.0013           38.27m
         2           1.6003           0.0013           36.18m
         3           1.6002           0.0014           35.78m
         4           1.6219           0.0013           35.57m
         5           1.5873           0.0013           35.23m
         6           1.6100           0.0013           35.24m
         7           1.6027           0.0014           35.43m
         8           1.5894           0.0014           35.28m
         9           1.6108           0.0013           35.20m
        10           1.6123           0.0013           35.14m
        20           1.5888           0.0013           37.17m
        30           1.5708           0.0013           44.96m
        40           1.5669           0.0013           43.73m
        50           1.5341           0.0012           41.90m
        60           1.5316           0.0012           41.24m
       

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.001, loss='ls', max_depth=3,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=5, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=2000,
             presort='auto', random_state=None, subsample=0.5, verbose=1,
             warm_start=False)

In [26]:
with open('gradient_boosting_regressor.pkl', 'wb') as f:
    # Write the model to a file.
    pickle.dump(gbr, f)

In [22]:
y_pred = gbr.predict(X_test)

In [23]:
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
print(rmse)

0.9889643083112633


In [ ]:
# with open('model.pkl', 'rb') as f:
#     model = pickle.load(f)